# Aim
We are creating a minimal implementation as a baseline to test the KGVAE model against. We will transform relevant portions of concept net, relevant to the ones we used for testing of the KGVAE model model, into an rdf style knowledge graph, and will generate outputs that we will use as a comparison. 

We will also make a proposal how testing on a larger scale could work.


In [574]:
import sys
!{sys.executable} -m pip install -q langchain langchain-community langchain-openai fastembed qdrant-client oxrdflib

from operator import itemgetter
import getpass
import os

from langchain.globals import set_debug
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import format_document
from langchain_openai import OpenAI
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import get_buffer_string
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Provide your OpenAI API Key")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [249]:
from rdflib import Graph, Namespace, URIRef, Literal# Generate new graph
from rdflib.namespace import FOAF, DCTERMS, XSD, RDF, SDO, RDFS

In [433]:
import requests
import re
from pprint import pprint

In [251]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download NLTK stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/ulli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ulli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [520]:
import pickle

In [252]:
import json

In [183]:
from tqdm import tqdm

# Set properties 

g.add((country_uri, RDF.type, SCHEMA.Country))
g.add((country_uri, RDFS.label, Literal('Netherlands', datatype=XSD.string)))
g.add((country_uri, CKO.iso2, Literal('NLD', datatype=XSD.string)))


observ_uri = URIRef(f"https://w3id.org/climatekg/observation/gdp/NLD/2021")
g.add((observ_uri, RDF.type, SCHEMA.Observation))
g.add((observ_uri, SCHEMA.about, country_uri))
g.add((observ_uri, SCHEMA.measuredProperty, CKO.GDP))
g.add((observ_uri, TIME.year, Literal(2021, datatype=XSD.gYear)))
g.add((observ_uri, SCHEMA.value, Literal(1.018007e+12, datatype=XSD.float)))

In [2]:
# Extract relevant entities from conceptnet

In [550]:
input_entities = [
    {  # Entity 1
        "sentences": [
            "The vast grasslands of the African savanna support diverse wildlife.",
            "African savannas are characterized by their grassy landscapes and scattered trees.",
            "Elephants, zebras, and giraffes are iconic herbivores of the African savanna.",
            "The seasonal fires on the savanna play a crucial role in maintaining the balance of the ecosystem.",
            "Droughts can be a significant challenge for animal and plant life in the African savanna.",
            "African savannas are found in tropical and subtropical regions.",
            "The savanna is characterized by alternating wet and dry seasons.",
            "Acacia trees are a common sight on the African savanna, their flat tops a distinctive shape.",
            "Tourism focused on wildlife viewing is an important economic activity for communities bordering the savanna.",
            "Overgrazing by livestock can threaten the delicate balance of the savanna ecosystem.",
            "Conservation efforts are vital to protect the unique biodiversity of the African savanna.",
        ],
        "questions": [
            "What is the savanna?",
            "Which animals live in the savanna?",
            "What is the climate in the savanna?"
        ]
         
    },
    {  # Entity 2
        "sentences": [
            "Animals are multicellular organisms.",
            "Animals can move independently in their environment.",
            "Animals have a wide variety of adaptations for survival.",
            "Animals range in size from microscopic organisms to the enormous blue whale.",
            "From insects in your backyard to deep-sea creatures, animals inhabit nearly every corner of the Earth.",
            "Animals can be classified as vertebrates (having a backbone) or invertebrates.",
            "Animals are heterotrophs, meaning they obtain their food by consuming other organisms.",
            "Animals can be herbivores (plant-eaters), carnivores (meat-eaters), or omnivores (eating both plants and animals).",
            "Some animals have specialized feeding structures like sharp teeth, beaks, or filter-feeding mechanisms.",
            "Animals reproduce either sexually or asexually.",
            "Animals exhibit diverse reproductive strategies, from laying eggs to giving birth to live young.",
            "Parental care varies greatly among animal species."
        ],
        "questions": [
            "How large are animals?",
            "What is a vertebrate?",
            "What do animals eat?"
        ]
    },
    {  # Entity 3
        "sentences": [
            "[[Renaissance art]] emphasized [[classical forms]] and [[humanism]].",
            "The intricate patterns of [[fractals]] reveal mathematical beauty.",
            "[[Ancient mythology]] provides insights into the beliefs of early [[civilizations]].",
            "Renaissance artists drew inspiration from the sculptures and architecture of ancient Greece and Rome.",
            "The study of classical philosophy and literature was a cornerstone of Renaissance humanism.",
            "Columns, arches, and symmetry are hallmarks of classical architectural forms.",
            "Humanism placed emphasis on the value of human potential and individual achievement.",
            "Renaissance thinkers questioned traditional authorities and focused on reason and observation.",
            "Humanists believed in the power of education to improve society.",
            "The self-similar patterns of fractals can be found in natural objects like ferns and snowflakes.",
            "Fractals are generated through mathematical formulas, revealing the hidden order within seemingly chaotic patterns.",
            "Fractals have applications in fields ranging from computer graphics to medicine. ",
            "Greek mythology is filled with stories of gods, heroes, and fantastical creatures.",
            "Myths often served to explain natural phenomena or provide moral lessons.",
            "Archaeological discoveries continue to uncover the secrets of ancient civilizations like Egypt and Mesopotamia. "
        ],
        "questions": [
            "Who inspired renaissance artists?",
            "What is a fractal?",
            "Who are gods in greek mythology?"
        ]
    },
    {  # Entity 4
        "sentences": [
           "The [[economic policies]] of the [[1980s]] had a lasting global impact.",
           "Name a large [[country]] in [[America]].",
           "Who was the 22nd president?",
           "The [[industrial revolution]] dramatically changed societies around the world.",
           "The deregulation movements of the 1980s led to significant shifts in the banking and telecommunications industries.",
           "In the United States, the 1980s are often remembered for the Reagan Administration's emphasis on free market principles.",
           "The fall of the Berlin Wall in 1989 symbolized the end of the Cold War era and opened the path to German reunification.",
           "Economic globalization accelerated in the 1980s, facilitated by advances in technology and reductions in trade barriers.",
           "The 22nd president of the United States, Grover Cleveland, served two non-consecutive terms, making him also the 24th president.",
           "Brazil is the largest country in South America by both area and population, playing a key role in regional politics and economics.",
           "The 'War on Drugs' policy initiated in the 1980s had profound effects on both domestic and international law enforcement strategies.",
           "The introduction of neoliberal policies in the 1980s impacted labor markets and social welfare systems worldwide.",
           "The Chernobyl disaster of 1986 raised global awareness about the potential dangers associated with nuclear power.",
           "The 1980s saw the emergence of the Internet from a research and military network to the beginnings of the global information system we know today.",
           "The economic policies of the 1980s, including tax cuts and deregulation, are often debated for their impact on income inequality.",
           "The space shuttle Challenger disaster in 1986 underscored the risks of space exploration and led to major safety reforms in NASA.",
           "The adoption of the Single European Act in 1986 was a significant step towards the creation of the European Single Market.",
           "The 1980s cultural landscape was marked by the rise of digital technology, influencing music, video games, and cinema.",
           "The environmental movement gained momentum in the 1980s, highlighted by the establishment of Earth Day as a global event for environmental awareness and protection.",
           "Reaganomics, with its focus on tax cuts and deregulation, was a dominant economic policy of the 1980s.",
           "The debate over Keynesian economics vs. supply-side economics shaped policy decisions during the 1980s.",
           "Rising national debt was a key economic issue facing the United States during the 1980s.",
           "The Cold War cast a long shadow over the geopolitical landscape of the 1980s.",
           "The development of personal computers and the rise of MTV mark significant cultural shifts in the 1980s.",
           "Fashion in the 1980s was often characterized by bright colors, big hair, and shoulder pads.",
           "Brazil is the largest country in South America.",
           "Canada shares the longest international border in the world with the United States.",
           "Diverse ecosystems, from the Amazon rainforest to the Andes Mountains, can be found across the countries of America.",
           "Grover Cleveland is the only president to serve two non-consecutive terms.",
           "The Louisiana Purchase was a major acquisition of land negotiated during Thomas Jefferson's presidency.",
           "Abraham Lincoln led the United States through the Civil War.",
           "The invention of the steam engine was a pivotal moment in the Industrial Revolution. ",
           "The Industrial Revolution led to mass migration from rural areas to urban centers. ",
           "New technologies developed during the Industrial Revolution transformed industries like textile production and transportation."
        ],
        "questions": [
            "What are economic issues in the 1980s?",
            "What are economic policied of the 1980s?",
            "What are the highest mountains in South America?"
        ]
    },
    {
    # Entity 5
    "sentences": [
       "What is a soft-fleeced animal related to the camel?",  # Alpaca or Llama
       "What is the chemical symbol for gold?",  # Au
       "Gold is a precious metal with high conductivity and resistance to corrosion.",
       "Alpacas are domesticated species of South American camelids, known for their soft wool.",
       "The llama, a close relative of the alpaca, is also used for its wool and as a pack animal.",
       "Gold has been used for coinage, jewelry, and other arts throughout recorded history.",
       "The density of gold is 19.32 grams per cubic centimeter, making it one of the densest elements.",
       "Alpacas and llamas are part of the Andean culture and economy, playing a significant role in traditional practices.",
       "Gold mining impacts environmental ecosystems through the release of toxic substances.",
       "The Incas valued alpacas for their wool and considered them a gift from Pachamama, the earth mother.",
       "Modern uses of gold include electronics, dentistry, and as an investment in the form of bullion and coins.",
       "Selective breeding of alpacas has been practiced for thousands of years to produce a variety of wool colors.",
       "Gold's atomic number is 79, and it is classified as a transition metal in the periodic table.",
       "Conservation efforts are in place to maintain the genetic diversity and health of domesticated alpaca populations.",
       "The extraction and processing of gold are energy-intensive and contribute to greenhouse gas emissions.",
    ],
        "questions": [
            "What does a llama eat?",
            "What is Gold used for?",
            "What are llamas used for?"
        ]
    },{
    # Entity 6
    "sentences": [
       "Deus sive Natura, meaning 'God or Nature', signifies the equivalence of God and Nature.",
       "This concept suggests that God is not a personal deity but the inherent order and essence of the universe.",
       "Originating from the philosophy of Baruch Spinoza, Deus sive Natura challenges traditional views of God.",
       "Spinoza's framework posits that everything that exists is a part of the singular substance, which is God or Nature.",
       "This perspective emphasizes the unity and interconnectedness of all things within the universe.",
       "Deus sive Natura implies that God's will is equivalent to the natural laws governing the universe.",
       "Spinoza’s philosophy promotes a pantheistic view, seeing God in all aspects of the natural world.",
       "The concept encourages a reverence for nature, seeing it as a manifestation of the divine.",
       "Deus sive Natura has influenced both theological and scientific thought, bridging spirituality with the natural sciences.",
       "Critics of this idea argue that it dilutes the concept of a personal, interactive God.",
       "Supporters find in Deus sive Natura a foundation for environmental ethics and a deeper understanding of our place in the universe.",
       "The notion has sparked debate regarding the nature of divinity, free will, and the meaning of sanctity.",
       "Deus sive Natura is considered a cornerstone in the development of modern philosophical naturalism.",
       "It challenges individuals to find spirituality within the laws and beauty of the natural world, rather than through supernatural beliefs.",
       "The concept has been explored in various forms of art and literature, reflecting humanity's complex relationship with the divine and the natural."
    ],
        "questions": [
            "What is the principle 'deus size natura'?",
            "What is a fractal?",
            "Who are gods in greek mythology?"
        ]
    },{
    # Entity 7
    "sentences": [
        "Mars, known as the Red Planet, is the fourth planet from the Sun in our solar system.",
        "Its distinctive red color is due to iron oxide, or rust, on its surface.",
        "Mars has the largest volcano in the solar system, Olympus Mons, and a vast canyon, Valles Marineris.",
        "The planet has two small moons, Phobos and Deimos, believed to be captured asteroids.",
        "Evidence suggests that Mars once had liquid water on its surface, indicating the potential for past life.",
        "Mars experiences dust storms, which can cover the entire planet and last for months.",
        "The thin atmosphere of Mars, composed mostly of carbon dioxide, is not suitable for human life.",
        "Mars has polar ice caps that contain frozen water and carbon dioxide, changing size with the seasons.",
        "NASA's rovers, such as Curiosity and Perseverance, have been exploring Mars to study its climate and geology.",
        "The concept of terraforming Mars has been proposed as a way to make the planet habitable for humans.",
        "Mars' gravity is about 38% of Earth's, affecting the potential for human muscle and bone density on the planet.",
        "Scientists believe that studying Mars can reveal much about the early history of our solar system.",
        "There is ongoing research into the possibility of past or present microbial life on Mars.",
        "Human missions to Mars are being planned, with the goal of sending astronauts within the next few decades.",
        "The exploration of Mars has been a significant part of popular culture, inspiring numerous works of science fiction."
    ],
        "questions": [
            "What is the atmosphere like on Mars?",
            "Is there life on Mars?",
            "Does Mars have moons?"
        ]
},{
    # Entity 8
    "sentences": [
        "Pizza dough is a foundational element of pizza, made from flour, water, yeast, and salt.",
        "The type of flour used, such as all-purpose or bread flour, can affect the texture and flavor of the dough.",
        "Yeast is essential for the fermentation process, which allows the dough to rise and develop its characteristic flavor.",
        "Kneading the dough is crucial for developing gluten, which gives the dough its elasticity and chewiness.",
        "Pizza dough can be prepared in advance and stored in the refrigerator to enhance its flavor through slow fermentation.",
        "The hydration ratio, or the amount of water relative to flour, is key to determining the dough's texture.",
        "Some pizza dough recipes include olive oil or sugar to enrich the flavor and color of the crust.",
        "Proper proofing, or allowing the dough to rise, is vital for achieving the desired volume and lightness.",
        "The thickness of the rolled-out dough can vary depending on the style of pizza, from thin crust to deep-dish.",
        "Baking pizza dough at a high temperature is important for creating a crisp exterior and soft interior.",
        "Sourdough pizza dough, made from a natural starter, offers a tangy flavor and rustic texture.",
        "The Neapolitan pizza dough, recognized for its simplicity and high-quality ingredients, adheres to strict guidelines for authenticity.",
        "Experimenting with different types of flour, such as whole wheat or gluten-free alternatives, can produce varied dough characteristics.",
        "The resting period after kneading and before shaping allows the gluten network to relax, making the dough easier to work with.",
        "The method of stretching the dough, whether by hand or rolling pin, affects the final texture and airiness of the crust."
    ],
        "questions": [
            "What goes on a pizza?",
            "How do you make pizza dough?",
            "What are ingredients for pizza dough?"
        ]
},{
    # Entity 9
    "sentences": [
        "Variational Autoencoders (VAEs) are a type of generative model in machine learning.",
        "They are used to generate new data points that are similar to the input data.",
        "VAEs consist of an encoder, a decoder, and a loss function that measures reconstruction error and the Kullback-Leibler divergence.",
        "The encoder converts input data into a smaller, dense representation in a latent space.",
        "The decoder then reconstructs the input data from this latent representation.",
        "VAEs are particularly known for their application in generating realistic images, texts, and other types of data.",
        "They work by learning a distribution of the input data in the latent space, allowing for the generation of new data points.",
        "One key feature of VAEs is their ability to perform dimensionality reduction, similar to PCA (Principal Component Analysis).",
        "The Kullback-Leibler divergence in the loss function encourages the latent space to follow a probabilistic distribution, typically a Gaussian.",
        "VAEs have been applied in various fields, including drug discovery, anomaly detection, and image super-resolution.",
        "They can also be used for semi-supervised learning, where the model learns from both labeled and unlabeled data.",
        "Unlike traditional autoencoders, VAEs can generate new data points, making them more versatile for tasks like image synthesis.",
        "The variational approach differentiates VAEs from other generative models by introducing a probabilistic interpretation of the encoder’s output.",
        "Training VAEs involves optimizing the balance between the reconstruction fidelity and the regularization of the latent space.",
        "The flexibility and efficiency of VAEs make them a powerful tool in the arsenal of machine learning practitioners and researchers."
    ],
        "questions": [
            "What is a Variational Autoencoder?",
            "Which loss function do you use for Variational Autoencoders?",
            "Which concepts do VAEs resemble?"
        ]
},{
    # Entity 10
    "sentences": [
       "Gubernatorial woman",
       "Who was the most sadistic figure in history?"
       "Gubernatorial woman refers to a female governor or a woman holding the office of a governor.",
       "Who was the most sadistic figure in history? This question explores individuals known for extreme cruelty or brutality throughout history.",
       "Female governors have played significant roles in the political landscapes of their regions, breaking gender barriers and setting precedents.",
       "Historical figures associated with sadism often include tyrants, dictators, and leaders noted for their ruthless policies or acts of violence.",
       "The rise of women in gubernatorial roles highlights the progress in gender equality within political leadership positions.",
       "Sadistic historical figures are frequently subjects of psychological studies to understand the motivations behind their actions.",
       "Gubernatorial women may also advocate for women's rights and work towards inclusive policies in their governance.",
       "Debates about the most sadistic figure in history involve considerations of context, power dynamics, and the impact of their actions over time.",
       "In recent years, there has been an increase in the number of women serving as governors across the world, reflecting a shift in societal norms.",
       "The legacy of sadistic figures in history is often marked by trauma, cultural memory, and the moral lessons drawn from their reigns.",
       "Female governors often face unique challenges and scrutiny in their roles, navigating both political and societal expectations.",
       "Studying sadistic figures in history can provide insights into the darker aspects of human nature and the conditions that enable such behavior.",
       "Gubernatorial women are often pioneers in their fields, setting examples for future generations of female leaders.",
       "The examination of historical cruelty requires a nuanced understanding of historical context, individual psychology, and societal conditions.",
       "The achievements of women in gubernatorial positions contribute to the broader dialogue on diversity and representation in leadership."

    ],
        "questions":[]
},{
    # Entity 11
    "sentences": [
       "The water buffalo (Bubalus bubalis) is a large bovid originating from South Asia, Southeast Asia, and China.",
       "Domesticated for thousands of years, water buffaloes are critical to agriculture in regions across Asia for plowing fields and irrigation.",
       "There are two main types of water buffalo: the river buffalo of South Asia and the swamp buffalo of Southeast Asia and China.",
       "Water buffaloes are known for their strength and ability to work in wet, marshy lands where other draft animals cannot.",
       "Apart from agricultural work, water buffaloes are also raised for their milk, meat, and hides.",
       "Water buffalo milk is a key ingredient in traditional cheeses in some countries, including mozzarella in Italy.",
       "These animals have a semi-aquatic lifestyle, spending a lot of their time submerged in water to cool off and avoid pests.",
       "Water buffaloes play a significant role in the ecosystems they inhabit, contributing to the biodiversity of wetlands.",
       "The global population of water buffaloes is estimated to be over 200 million, with a significant number in India and China.",
       "Conservation efforts are in place for wild populations of water buffalo, which are threatened by habitat loss and hunting.",
       "Water buffaloes have a social structure, typically forming herds led by a dominant female.",
       "The gestation period for a water buffalo is about 10 to 11 months, with calves being born with a distinctive reddish-brown coat.",
       "They are known for their large, crescent-shaped horns, which can extend up to 2 meters in some individuals.",
       "Water buffaloes have been introduced to other parts of the world, including Australia, for farming and weed control.",
       "Cultural and religious significance of the water buffalo varies across different societies, symbolizing strength, agriculture, and prosperity."
    ],
        "questions": [
            "Where does the water buffalo originate from?",
            "What is the water buffalo used for?",
            "What does the water buffalo signify?"
        ]
},{
    # Entity 12
    "sentences": [
        "The Marshall Islands is a country located in the central Pacific Ocean, part of the larger island group of Micronesia.",
        "It consists of 29 atolls and 5 individual islands, totaling over 1,000 individual islets and islands.",
        "The capital of the Marshall Islands is Majuro, which also serves as the main commercial and administrative center.",
        "The Marshall Islands gained independence from the United States in 1986 under the Compact of Free Association.",
        "The islands were significantly impacted by nuclear testing conducted by the United States during the 1940s and 1950s.",
        "The Marshall Islands is known for its vibrant marine life and is a popular destination for scuba diving and snorkeling.",
        "Due to its low elevation, the Marshall Islands is highly vulnerable to the effects of climate change, including sea level rise.",
        "The Marshallese culture is rich in maritime tradition, reflecting the inhabitants' deep connection to the ocean and navigation skills.",
        "The economy of the Marshall Islands is supported by the use of its large ship registry and financial assistance from the United States.",
        "Marshallese is the official language, with English also widely spoken, especially in official settings and education.",
        "The Bikini Atoll, part of the Marshall Islands, is a UNESCO World Heritage Site, recognized for its underwater nuclear test sites.",
        "The Marshall Islands has taken a proactive stance in international forums on issues related to nuclear disarmament and climate change.",
        "Fishing and coconut processing represent important parts of the local economy, alongside growing tourism.",
        "The traditional navigation techniques of the Marshallese, using stick charts and star maps, have gained international recognition.",
        "The Marshall Islands faces challenges such as limited natural resources, reliance on imports, and the need for sustainable development."
    ],
        "questions": [
            "Where are the Marshall Islands?",
            "Why are the Marshall Islands significant?",
            "What does the economy of the Marshall Islands rely on?"
        ]
},{
    # Entity 13
    "sentences": [
        "Tristan da Cunha is the most remote inhabited archipelago in the world, located in the South Atlantic Ocean.",
        "It is part of the British Overseas Territory, which also includes Saint Helena and Ascension Island.",
        "The main island of the archipelago is also named Tristan da Cunha and is home to a small population.",
        "Tristan da Cunha is known for its unique ecosystem, with many species that are endemic to the island.",
        "The island is accessible only by sea, with trips from South Africa taking about six days.",
        "The economy of Tristan da Cunha is based on fishing, agriculture, and the sale of postage stamps to collectors.",
        "Due to its remote location, the community on Tristan da Cunha is largely self-sufficient.",
        "The archipelago was discovered in 1506 by Portuguese explorer Tristão da Cunha, but it was not permanently settled until the 19th century.",
        "Tristan da Cunha has an active volcano, Queen Mary's Peak, which last erupted in 1961, leading to the temporary evacuation of the island's inhabitants.",
        "The islanders of Tristan da Cunha have a unique dialect, which is a blend of English, Dutch, and other languages.",
        "Conservation efforts are in place to protect the island's wildlife, including the Tristan albatross and the rockhopper penguins.",
        "Tourism is limited due to the island's remote location, but it offers unique opportunities for eco-tourism and adventure travel.",
        "Tristan da Cunha's isolation has made it a subject of interest for researchers studying biodiversity and isolation's effects on communities.",
        "The island has no airport, making the sea the only way to access this remote community.",
        "The residents of Tristan da Cunha live in the settlement of Edinburgh of the Seven Seas, often referred to as the most remote settlement in the world."
    ], 
        "questions": [
            "Where is Tristan da Cunha?",
            "What is Tristan da Cunha?",
            "What does the economy of Tristan da Cunha rely on?"
        ]
}


]


In [531]:
all_text = [y for x in input_entities for y in x['sentences']]

In [503]:
punctuation = set(string.punctuation)

# Get the list of stopwords
stop_words = set(stopwords.words('english'))

# List to store words without stopwords
words_without_stopwords = []

# Tokenize each sentence and remove stopwords
for sentence in all_text:
    # Tokenize the sentence
    words = word_tokenize(sentence)
    # Remove stopwords from the tokenized words
    words_without_stopwords.extend([word.lower() for word in words if word.lower() not in stop_words and word not in punctuation])

In [510]:
def get_conceptnet_data(concept):
    url = f"http://api.conceptnet.io/c/en/{concept}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to retrieve data from ConceptNet API")
        return None

In [505]:
def concept_to_graph(concept_data):
    CN  = Namespace("https://conceptnet.io/")
    g = Graph()
    g.bind("cn", CN)

    for a, v in concept_data.items():
        if a == "@id":
            (_, _, lang, name) = v.split("/")
        if a == "edges":
            edges = v

            for entry in edges:

                m = re.search(r"\[(?:/(?P<rtype>.)/(?P<rname>[^/]*)/.*),(?:/./(?P<flang>..)/(?P<flabel>[^/]*?)/.*),(?:/./(?P<tlang>..)/(?P<tlabel>[^/]*?)/.*)\]", entry["@id"])

                if m != None:
                    rtype, rname, flang, flabel, tlang, tlabel = m.groups()
                    if flang == tlang and flang == "en": 

                        if all(key in entry for key in ['start', 'end', 'rel']):
                            startid = entry['start']["@id"]
                            startlabel = entry['start']["label"]
                            starttype = entry['start']["@type"]
                            endid = entry['end']["@id"]
                            endlabel = entry['end']["label"]
                            endtype = entry['end']["@type"]
                            relid = entry['rel']["@id"]
                            rellabel = entry['rel']["label"]
                            reltype = entry['rel']["@type"]

                            starturi = CN[startid]
                            g.add((starturi, RDFS.label, Literal(startlabel, datatype=XSD.string)))
                            enduri = CN[endid]
                            g.add((enduri, RDFS.label, Literal(endlabel, datatype=XSD.string)))

                            reluri = CN[relid]
                            g.add((reluri, RDFS.label, Literal(rellabel, datatype=XSD.string)))

                            g.add((starturi, reluri, enduri))
                        if "surfaceText" in entry and entry["surfaceText"] != None:
                            surfaceText = Literal(entry['surfaceText'].replace("]", "").replace("[", "").replace("*", ""), lang='en')
                            g.add((starturi, CN.hasSurfaceText, surfaceText))
    return g


In [163]:
g = concept_to_graph(concept_data)

In [506]:
get_conceptnet_data("spinoza")

{'@context': ['http://api.conceptnet.io/ld/conceptnet5.7/context.ld.json'],
 '@id': '/c/en/spinoza',
 'edges': [{'@id': '/a/[/r/Synonym/,/c/en/spinoza/n/wn/person/,/c/en/benedict_de_spinoza/n/wn/person/]',
   '@type': 'Edge',
   'dataset': '/d/wordnet/3.1',
   'end': {'@id': '/c/en/benedict_de_spinoza/n/wn/person',
    '@type': 'Node',
    'label': 'Benedict de Spinoza',
    'language': 'en',
    'sense_label': 'n, person',
    'term': '/c/en/benedict_de_spinoza'},
   'license': 'cc:by/4.0',
   'rel': {'@id': '/r/Synonym', '@type': 'Relation', 'label': 'Synonym'},
   'sources': [{'@id': '/s/resource/wordnet/rdf/3.1',
     '@type': 'Source',
     'contributor': '/s/resource/wordnet/rdf/3.1'}],
   'start': {'@id': '/c/en/spinoza/n/wn/person',
    '@type': 'Node',
    'label': 'Spinoza',
    'language': 'en',
    'sense_label': 'n, person',
    'term': '/c/en/spinoza'},
   'surfaceText': '[[Spinoza]] is a synonym of [[Benedict de Spinoza]]',
   'weight': 2.0},
  {'@id': '/a/[/r/DerivedFro

In [511]:
all_concepts = Graph()
for i,word in enumerate(set(words_without_stopwords)):
    print(i,word)
    try:
        cdata = get_conceptnet_data(word)
        g = concept_to_graph(cdata)
    except:
        print(f"{word} could not be retrieved")
    all_concepts += g

0 cover
1 andean
2 among
3 enrich
4 pin
5 airiness
6 achievement
7 jewelry
8 strength
9 holding
10 plant-eaters
11 1961
12 last
13 isolation
14 feeding
15 pantheistic
16 arsenal
17 milk
18 strategies
19 due
20 president
21 albatross
22 valles
23 emphasized
24 societal
25 environment
26 tristão
27 ocean
28 proactive
29 precedents
30 oil
31 focused
32 phenomena
33 trade
34 gods
35 expectations
36 highlighted
37 resting
38 initiated
39 global
40 generate
41 dominant
42 highlights
43 introduced
44 camelids
45 beginnings
46 islands
47 fractals
48 supporters
49 theological
50 microscopic
51 11
52 regions
53 community
54 momentum
55 around
56 feature
57 filled
58 used
59 marshallese
60 support
61 reliance
62 images
63 challenger
64 salt
65 sourdough
66 find
67 muscle
68 violence
69 reddish-brown
70 religious
71 widely
72 reproduce
73 refrigerator
74 psychological
75 recent
76 learns
77 early
78 supply-side
79 facing
80 spirituality
81 earth
82 snowflakes
83 sexually
84 spinoza
85 calves
86 fe

672 social
673 hand
674 aspects
675 fields
676 compact
677 commercial
678 snorkeling
679 stick
680 maps
681 significance
682 space
683 humans
684 implies
685 testing
686 include
687 majuro
688 agricultural
689 wild
690 rather
691 pack
692 heterotrophs
693 type
694 connection
695 10
696 significantly
697 every
698 lasting
699 terms
700 densest
701 provides
702 steam
703 animals
704 heroes
705 enforcement
706 god
707 broader
708 herbivores
709 posits
710 flour
711 animal
712 all-purpose
713 contribute
714 range
715 columns
716 extreme
717 chewiness
718 human
719 movement
720 suggests
721 proper
722 highly
723 independently
724 discoveries
725 geology
726 archipelago
727 overseas
728 technology
729 name
730 involves
731 laws
732 1989
733 specialized
734 1980s
735 intricate
736 rockhopper
737 present
738 territory
739 south
740 populations
741 accelerated
742 climate
743 alpaca
744 location
745 tax
746 decisions
747 deus
748 labeled
749 kneading
750 atoll
751 generation
752 resistance
753 

In [512]:
all_concepts.serialize("all_concepts.ttl", format='turtle')

<Graph identifier=N1f999e1145464058b22ca76ad7c31032 (<class 'rdflib.graph.Graph'>)>

In [514]:
len(all_concepts)

23852

## ⚗️ Define how to extract infos to embed from the ontology

We will extend the LangChain document loader class to create a loader for OWL ontologies

**TODO**: you will need to write the SPARQL query to extract the concepts (classes and properties) labels and descriptions from the ontology. 

The current loader code expect your query to return the variables below.
- `?uri` of the concept
- `?label` containing the human readable label or description of this concept
- `?predicate` defining the retrieved label (e.g. rdfs:label, rdfs:comment, dcterms:description)
- `?type` of the concept (e.g. owl:Class or property)

Feel free to add more variables! You will just need to add 1 line to the code below to add this variable as concept metadata in the vectorstore

In [515]:
from typing import Any, List, Optional

from langchain_community.document_loaders.base import BaseLoader
from langchain_core.documents import Document
from rdflib import Graph

# TODO: the SPARQL query to retrieve labels and descriptions of classes and properties
extract_classes_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX cn: <https://conceptnet.io/>
SELECT * WHERE {
  ?sub ?pred ?obj ;
       rdfs:label ?sublabel .
  ?obj rdfs:label ?objlabel .
  OPTIONAL { ?sub cn:hasSurfaceText ?text}
}
"""

class OntologyLoader(BaseLoader):
    """Load an OWL ontology and extract classes and properties as documents."""

    def __init__(self, ontology_url: str, format: Optional[str] = None):
        self.ontology_url = ontology_url
        self.format = format
        self.graph = Graph(store="Oxigraph")
        self.graph.parse(source=self.ontology_url, format=self.format)

    def load(self) -> List[Document]:
        """Load and return documents (classes and properties) from the OWL ontology."""
        docs: List[Document] = []
        for cls in self.graph.query(extract_classes_query):
            docs.append(self._create_document(cls))
        return docs

    def _create_document(self, result_row: Any) -> Document:
        """Create a Document object from a query result row."""
        label = str(result_row.sub)
        metadata={
                "sub": str(result_row.sublabel),
                "uri": str(result_row.sub),

                "pred": str(result_row.pred),
                "predicate": str(result_row.pred),

                "obj": str(result_row.objlabel),
                "type": str(result_row.obj),

                "ontology": self.ontology_url,
            }
        if "text" in result_row.asdict():
            metadata.update({"surfaceText": result_row.text})
        else:
            metadata.update({"surfaceText": ""})
            

        return Document(
            page_content=label,
            # NOTE: you can include more metadata retrieved by the SPARQL query here
            metadata=metadata
        )


## 🌀 Initialize local vectorstore and LLM

# This takes approx 10-15 minutes to run

In [518]:
flag_embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-small-en-v1.5", max_length=512)
loader = OntologyLoader("all_concepts.ttl", format="turtle")
docs = loader.load()

# Split the documents into chunks if necessary
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Qdrant.from_documents(
    splits,
    flag_embeddings,
    collection_name="ontologies",
    location=":memory:",
#     path="./data/qdrant"
)

Fetching 9 files: 100%|████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 37560.93it/s]


In [532]:
# vectorstore = FAISS.from_documents(documents=docs, embedding=flag_embeddings)
# K is the number of source documents retrieved
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

llm = OpenAI(temperature=0)

## 🧠 Initialize prompts and memory

In [575]:

# Create the memory object that is used to add messages
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)
# Add a "memory" key to the input object
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Prompt to reformulate the question using the chat history
reform_template = """Given the following chat history and a follow up question,
rephrase the follow up question to be a standalone straightforward question, in its original language.
Make sure it is not a yes/no question.
Do not answer the question! Just rephrase reusing informations from the chat history.
Make it short and straight to the point.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
REFORM_QUESTION_PROMPT = PromptTemplate.from_template(reform_template)

# Prompt to ask to answer the reformulated question
answer_template = """Briefly answer the question based only on the following context,
do not use any information outside this context.
The answer should be a sequence of a declarative clauses.
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(
    template="Concept label: {page_content} | URI: {uri} | Type: {type} | Subject:{sub} | Predicate: {predicate} | Object: {obj} | Text:{surfaceText} | Ontology: {ontology}"
)
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    # print("Formatted docs:", doc_strings)
    return document_separator.join(doc_strings)


## ⛓️ Define the chain

`itemgetter()` is used to retrieve objects defined in the previous step in the chain.

In [576]:
# Reformulate the question using chat history
reformulated_question = {
    "reformulated_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(x["chat_history"]),
    }
    | REFORM_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Retrieve the documents using the reformulated question
retrieved_documents = {
    "docs": itemgetter("reformulated_question") | retriever,
    "question": lambda x: (print("💭 Reformulated questions:", x["reformulated_question"]),
                               pprint(vectorstore.similarity_search_with_relevance_scores(x['reformulated_question'])[0]), 
                               pprint(vectorstore.similarity_search_with_score(x['reformulated_question'])[0])) or x["reformulated_question"],
    # "question": lambda x: x["reformulated_question"],
}
# Construct the inputs for the final prompt using retrieved documents
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# Generate the answer using the retrieved documents and answer prompt
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# Put the chain together
final_chain = loaded_memory | reformulated_question | retrieved_documents | answer

def stream_chain(final_chain, memory: ConversationBufferMemory, inputs: dict[str, str]) -> dict[str, Any]:
    """Ask question, stream the answer output, and return the answer with source documents."""
    output = {"answer": ""}
    for chunk in final_chain.stream(inputs):
#         print(f"{chunk=}", flush=True)
        if "docs" in chunk:
            output["docs"] = [doc.dict() for doc in chunk["docs"]]
            print("📚 Documents retrieved:")
            for doc in output["docs"][0:1]:
                print(f"· {doc['page_content']} ({doc['metadata']['uri']})")
#             print(json.dumps(output["docs"], indent=2))
        if "answer" in chunk:
            output["answer"] += chunk["answer"]
            print(chunk["answer"], end="", flush=True)
    # Add messages to chat history
    memory.save_context(inputs, {"answer": output["answer"]})
    return output

## 🗨️ Ask questions

In [577]:
questions = [y for x in input_entities for y in x['questions']]

In [578]:
for query in questions:
    print()
    print(query)
    print()
    output = stream_chain(final_chain, memory, {
    "question": query
})
    print()


What is the savanna?

💭 Reformulated questions:  Can you explain what the savanna is?
(Document(page_content='https://conceptnet.io//c/en/savannas/n', metadata={'sub': 'savannas', 'uri': 'https://conceptnet.io//c/en/savannas/n', 'pred': 'https://conceptnet.io//r/FormOf', 'predicate': 'https://conceptnet.io//r/FormOf', 'obj': 'savanna', 'type': 'https://conceptnet.io//c/en/savanna', 'ontology': 'all_concepts.ttl', 'surfaceText': '', '_id': 'd0dccd6d89aa4139b52d54410e78e4c5', '_collection_name': 'ontologies'}),
 0.7254349223971348)
(Document(page_content='https://conceptnet.io//c/en/savannas/n', metadata={'sub': 'savannas', 'uri': 'https://conceptnet.io//c/en/savannas/n', 'pred': 'https://conceptnet.io//r/FormOf', 'predicate': 'https://conceptnet.io//r/FormOf', 'obj': 'savanna', 'type': 'https://conceptnet.io//c/en/savanna', 'ontology': 'all_concepts.ttl', 'surfaceText': '', '_id': 'd0dccd6d89aa4139b52d54410e78e4c5', '_collection_name': 'ontologies'}),
 0.7254349223971348)
📚 Documents r

📚 Documents retrieved:
· https://conceptnet.io//c/en/renaissance/n/wn/act (https://conceptnet.io//c/en/renaissance/n/wn/act)

Artists are capable of painting portraits and letter signs, as well as coating a canvas with paint. They are also considered crazy and can be found at gallery openings.

What is a fractal?

💭 Reformulated questions:  Can you explain what a fractal is?
(Document(page_content='https://conceptnet.io//c/en/fractals/n', metadata={'sub': 'fractals', 'uri': 'https://conceptnet.io//c/en/fractals/n', 'pred': 'https://conceptnet.io//r/FormOf', 'predicate': 'https://conceptnet.io//r/FormOf', 'obj': 'fractal', 'type': 'https://conceptnet.io//c/en/fractal', 'ontology': 'all_concepts.ttl', 'surfaceText': '', '_id': '838ea50b8a8649bfbd9f5326cbecdcfa', '_collection_name': 'ontologies'}),
 0.7466395335872522)
(Document(page_content='https://conceptnet.io//c/en/fractals/n', metadata={'sub': 'fractals', 'uri': 'https://conceptnet.io//c/en/fractals/n', 'pred': 'https://conceptnet.i


Llamas are domesticated animals. They have a nasty habit of spitting.

What is Gold used for?

💭 Reformulated questions:  What is the purpose of gold?
(Document(page_content='https://conceptnet.io//c/en/gold', metadata={'sub': 'gold', 'uri': 'https://conceptnet.io//c/en/gold', 'pred': 'https://conceptnet.io//r/IsA', 'predicate': 'https://conceptnet.io//r/IsA', 'obj': 'a precious metal', 'type': 'https://conceptnet.io//c/en/precious_metal', 'ontology': 'all_concepts.ttl', 'surfaceText': rdflib.term.Literal('gold is related to precious', lang='en'), '_id': '53aa2fba62c640f1a8a6383d17c480d0', '_collection_name': 'ontologies'}),
 0.6381122701819781)
(Document(page_content='https://conceptnet.io//c/en/gold', metadata={'sub': 'gold', 'uri': 'https://conceptnet.io//c/en/gold', 'pred': 'https://conceptnet.io//r/IsA', 'predicate': 'https://conceptnet.io//r/IsA', 'obj': 'a precious metal', 'type': 'https://conceptnet.io//c/en/precious_metal', 'ontology': 'all_concepts.ttl', 'surfaceText': rdfli

(Document(page_content='https://conceptnet.io//c/en/mars', metadata={'sub': 'mars', 'uri': 'https://conceptnet.io//c/en/mars', 'pred': 'https://conceptnet.io//r/IsA', 'predicate': 'https://conceptnet.io//r/IsA', 'obj': 'planet', 'type': 'https://conceptnet.io//c/en/planet', 'ontology': 'all_concepts.ttl', 'surfaceText': rdflib.term.Literal('Mars is a planet', lang='en'), '_id': '8448d30550204052a3b8701c1b309173', '_collection_name': 'ontologies'}),
 0.6220474600334637)
📚 Documents retrieved:
· https://conceptnet.io//c/en/mars (https://conceptnet.io//c/en/mars)

Mars is a planet.

Does Mars have moons?

💭 Reformulated questions:  What are the moons of Mars?
(Document(page_content='https://conceptnet.io//c/en/moons', metadata={'sub': 'moons', 'uri': 'https://conceptnet.io//c/en/moons', 'pred': 'https://conceptnet.io//r/FormOf', 'predicate': 'https://conceptnet.io//r/FormOf', 'obj': 'moon', 'type': 'https://conceptnet.io//c/en/moon/n', 'ontology': 'all_concepts.ttl', 'surfaceText': '', '_


Autoencoders are derived from encoders, which are derived from variations and regularizations.

Which concepts do VAEs resemble?

💭 Reformulated questions:  What concepts are VAEs similar to?
(Document(page_content='https://conceptnet.io//c/en/vaesite', metadata={'sub': 'vaesite', 'uri': 'https://conceptnet.io//c/en/vaesite', 'pred': 'https://conceptnet.io//r/DerivedFrom', 'predicate': 'https://conceptnet.io//r/DerivedFrom', 'obj': 'vaes', 'type': 'https://conceptnet.io//c/en/vaes', 'ontology': 'all_concepts.ttl', 'surfaceText': '', '_id': '2ca4475508ac4b95a9bb75bbe0567b43', '_collection_name': 'ontologies'}),
 0.7431658348693905)
(Document(page_content='https://conceptnet.io//c/en/vaesite', metadata={'sub': 'vaesite', 'uri': 'https://conceptnet.io//c/en/vaesite', 'pred': 'https://conceptnet.io//r/DerivedFrom', 'predicate': 'https://conceptnet.io//r/DerivedFrom', 'obj': 'vaes', 'type': 'https://conceptnet.io//c/en/vaes', 'ontology': 'all_concepts.ttl', 'surfaceText': '', '_id': '2ca44

(Document(page_content='https://conceptnet.io//c/en/marshall_islands/n/wn/location', metadata={'sub': 'Marshall Islands', 'uri': 'https://conceptnet.io//c/en/marshall_islands/n/wn/location', 'pred': 'https://conceptnet.io//r/PartOf', 'predicate': 'https://conceptnet.io//r/PartOf', 'obj': 'Micronesia', 'type': 'https://conceptnet.io//c/en/micronesia/n/wn/location', 'ontology': 'all_concepts.ttl', 'surfaceText': rdflib.term.Literal('Marshall Islands is a part of Micronesia', lang='en'), '_id': 'c775e2eb3afe4abb8eaa31d13dc41d64', '_collection_name': 'ontologies'}),
 0.7127973524565933)
📚 Documents retrieved:
· https://conceptnet.io//c/en/marshall_islands/n/wn/location (https://conceptnet.io//c/en/marshall_islands/n/wn/location)

The Marshall Islands is a part of Micronesia. South Sea Islands is a type of island. Atolls are related to atoll and are a form of atoll. Gilbert and Ellice Islands is a part of Micronesia.

What does the economy of the Marshall Islands rely on?

💭 Reformulated qu